# VGG Configs

Ref: https://www.researchgate.net/publication/326989744_Unconventional_Wisdom_A_New_Transfer_Learning_Approach_Applied_to_Bengali_Numeral_Classification

In [ ]:
# model with ALL layers frozen, except for the SOFTMAX layer
def get_model_exp1_a():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax')(X)
    # magical line of freezing layers
    for layer in base_model.layers:
        layer.trainable=False
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

model=get_model_exp1_a()
model.summary()

In [ ]:
# model with all layers UNFROZEN, GPU might explode!
def get_model_exp1_b():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax')(X)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

model=get_model_exp1_b()
model.summary()

In [ ]:
# model with only the SOFTMAX layer frozen, unconventional wisdom kicks in!
def get_model_exp2_a():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax',trainable=False)(X)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

model=get_model_exp2_a()
model.summary()

In [ ]:
# model with SOFTMAX layer and last 4 vgg layers frozen, others layers remain unfrozen unconventional wisdom for the win!
def get_model_exp2_b():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax',trainable=False)(X)
    for layer in base_model.layers[-4:]:
        layer.trainable=False
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

model=get_model_exp2_b()
model.summary()

In [ ]:
def get_model_exp2_c():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    # last layer unfrozen here
    predictions = Dense(10, activation='softmax',trainable=True)(X)
    # intermediate layers fronzen
    for layer in base_model.layers[-5:]:
        layer.trainable=False
    #vgg divided in two parts, trained jointly
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

model=get_model_exp2_c()
model.summary()

In [ ]:
# model with only the SOFTMAX layer frozen, unconventional wisdom kicks in!
def get_model_exp2_d():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(48, 48, 3))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(10, activation='softmax')(X)
    #for layer in base_model.layers[::2]:
    #        layer.trainable = False
    
    model = Model(inputs=base_model.input, outputs=predictions)
    weights = model.layers[-1].get_weights()
   
    w1 = np.zeros( weights[0].shape )
    w2 = np.zeros( weights[1].shape )
    w  = [w1,w2]
    for i in range(0,10):
        w1[i*50:(i+1)*50,i] = 1.0
    print(w1.shape)
    model.layers[-1].set_weights( w)
    print(w2)
    model.layers[-1].trainable=False
    
    
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    #model.summary()
    return model

model=get_model_exp2_d()
model.summary()

# Data generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


model=get_model_exp2_c()
path_model='mnist2c-dg.h5' 


#model.load_weights(path_model)
K.set_value(model.optimizer.lr,1e-2) # set the learning rate

datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=35,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range = 20,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(rgb_arr_to_3d)

model.fit_generator(datagen.flow(rgb_arr_to_3d, Y_train, batch_size=128),
                    steps_per_epoch=len(rgb_arr_to_3d) / 128, 
                    epochs=100, 
                    verbose=1, 
                    validation_data=(rgb_arr_to_3d_test,Y_test),
                    callbacks=[
                       ModelCheckpoint(filepath=path_model, monitor='val_acc',  save_best_only=False),
                    ]      
                   )
